## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [2]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "THE CJ CUP Byron Nelson"
tournament_date = "5/4/2025"
old_course = "TPC Craig Ranch"
tournament_id = "R2025019"

# New Tournament
new_tournament_name = "Truist Championship"
new_course = "The Philadelphia Cricket Club"

In [3]:
# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os

# ====================

tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

### Update Database

#### Old Tournament

In [5]:
from utils.db_utils import update_tournament_results

# Define your constants
season = 2025
year = 20250
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")

# Run the update
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Preview the result
tournDf.head()



SSLError: HTTPSConnectionPool(host='orchestrator.pgatour.com', port=443): Max retries exceeded with url: /graphql (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))

In [4]:
import requests

X_API_KEY = "da2-gsrx5bibzbb4njvhl7t37wqyl4"

test_payload = {
    "operationName": "TournamentPastResults",
    "variables": {
        "tournamentPastResultsId": "R2025012",  # Example tournament ID
        "year": 20250
    },
    "query": "query TournamentPastResults($tournamentPastResultsId: ID!, $year: Int) {\n  tournamentPastResults(id: $tournamentPastResultsId, year: $year) {\n    id\n    players { id position }\n  }\n}"
}

try:
    r = requests.post(
        "https://orchestrator.pgatour.com/graphql",
        json=test_payload,
        headers={"x-api-key": X_API_KEY},
        verify=False  # TEMP bypass
    )
    r.raise_for_status()
    print("✅ API response received!")
    print("Response keys:", r.json().keys())
except Exception as e:
    print("❌ API test failed:")
    print(e)

c:\Users\ryanh\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'orchestrator.pgatour.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ API response received!
Response keys: dict_keys(['data'])
